In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import numpy as np

In [ ]:
import xarray as xr

# Base URL for the THREDDS OPeNDAP server (note: use "dodsC" instead of "fileServer")
base_url = "https://opendap.4tu.nl/thredds/dodsC/data2/djht/f359cd0f-d135-416c-9118-e79dccba57b9/1/2021"

# Generate the list of monthly URLs for the year 2021
urls = [
    f"{base_url}/TU-MREL_EU_ATL-2M_2021{month:02d}.nc"
    for month in range(1, 13)
]

# Load and combine all NetCDF files along the time axis
ds = xr.open_mfdataset(urls, combine='by_coords')

# Preview dataset structure: variables, dimensions, coordinates
print(ds)


<xarray.Dataset> Size: 49GB
Dimensions:    (latitude: 834, longitude: 734, time: 744)
Coordinates:
  * longitude  (longitude) float32 3kB -12.0 -11.97 -11.94 ... 9.93 9.96 9.99
  * latitude   (latitude) float32 3kB 36.0 36.03 36.06 ... 60.93 60.96 60.99
  * time       (time) datetime64[ns] 6kB 2021-01-01 ... 2021-01-31T23:00:00
Data variables: (12/29)
    crs        |S64 64B ...
    MAPSTA     (latitude, longitude) int16 1MB ...
    dpt        (time, latitude, longitude) float32 2GB ...
    ucur       (time, latitude, longitude) float32 2GB ...
    vcur       (time, latitude, longitude) float32 2GB ...
    uwnd       (time, latitude, longitude) float32 2GB ...
    ...         ...
    pdir0      (time, latitude, longitude) float32 2GB ...
    pdir1      (time, latitude, longitude) float32 2GB ...
    pdir2      (time, latitude, longitude) float32 2GB ...
    pdp0       (time, latitude, longitude) float32 2GB ...
    pdp1       (time, latitude, longitude) float32 2GB ...
    pdp2       (

In [ ]:
# Load EU NUTS2 data (excluding GB)
nuts2_eu_path = 'NUTS_RG_60M_2024_3035.geojson'
nuts2_eu_gdf = gpd.read_file(nuts2_eu_path)
nuts2_eu_gdf = nuts2_eu_gdf[nuts2_eu_gdf['LEVL_CODE'] == 2]  # Keep only NUTS2 regions

# Load GB-specific NUTS2 data
nuts2_gb_path = 'NUTS_RG_60M_2021_3035.geojson'
nuts2_gb_gdf = gpd.read_file(nuts2_gb_path)
nuts2_gb_gdf = nuts2_gb_gdf[nuts2_gb_gdf['LEVL_CODE'] == 2]  # Keep only NUTS2 regions 
nuts2_gb_gdf = nuts2_gb_gdf[nuts2_gb_gdf['CNTR_CODE'] == 'UK']

# Merge EU and GB NUTS2 regions
nuts2_gdf = pd.concat([nuts2_eu_gdf, nuts2_gb_gdf], ignore_index=True)
nuts2_gdf = nuts2_gdf.drop_duplicates(subset=['NUTS_ID'])
nuts2_gdf